In [93]:
# import Taking_Grams
# cp=Taking_Grams.getGrams(['8g1'])
from decimal import *
import pymongo
from pymongo import MongoClient
# cp=['C', 'Bdim']#產生的調
cp=['C', 'D', 'Bb', 'E', 'C', 'Bdim', 'Bb', 'E']#產生的調
dic_cp = {'C':[0, 4, 7],'C7':[0,4,7,10],'Cmaj7':[0, 4, 7, 11],
    'Dm':[2, 5, 9],'D':[2,6,9],'Dm7':[2, 5, 9, 12],'D7':[2,6,9,12],
	'E':[4,8,11],'E7':[4,8,11,14],'Em':[4, 7, 11],'Em7':[4, 7, 11, 14],
    'F':[5, 9, 12],'Fmaj7':[5, 9, 12, 16],
    'G':[7, 11, 14],'G7':[7, 11, 14, 17],
	'A':[9,13,16],'Am':[9, 12, 16],'Am7':[9, 12, 16, 19],
    'Bb':[10, 14, 17],'Bdim':[11, 14, 17]}
client = MongoClient('mongodb://10.120.30.8:27017')
db = client['music']  
collect = db['guitar_pattern_with_track_A_pattern']
cur = collect.find({'duration_count':1})
# cur = collect.find()

# aa=[{u'duration_count': 0.75, u'_id': 2, u'A_pattern': {u'track0': u'0.75,1;0.25,1,1'}},
# {u'duration_count': 1.0, u'_id': 4, u'A_pattern': {u'track0': u'0.125,1;0.125,1,0;0.125,1,0;0.125,1,0;0.125,1,0;0.125,1,0;0.125,1,0;0.125,1,0'}},
# {u'duration_count': 1.0, u'_id': 5, u'A_pattern': {u'track0': u'0.125,1;0.125,1,1;0.125,1,1;0.125,1,0;0.125,1,1;0.125,1,1;0.125,1,0;0.125,1,0'}},
# {u'duration_count': 1.0, u'_id': 7, u'A_pattern': {u'track0': u'0.125,0;0.125,1;0.25,0;0.5,1,0', u'track17': u'0.25,1;0.125,0;0.125,1,0;0.375,1,0;0.0625,1,0;0.0625,0'}},]
k=0
# for a in aa[0:4]:
for a in cur:   
    aa_i=a[u'A_pattern']#第某個dic的 A_pattern:{track,track2}        
    for key in aa_i:
        j=k
        if 'track' in key:   #將A_pattern中的KEY分別判斷有沒有track
            duration=0#拍子長度
            i=0 #和弦升降值
            A=[]
            durationType_pitch='' 
            for pitch in aa_i[key].split(';'):  #track用分號切割成每個Pitch            
                if len(pitch.split(','))==2:#沒有升降值的給升降值0,pitch就有[duration,有無音,升降值]
                    pitch=pitch+',0'
                if duration>=0.5:    #如果duration超過0.5,新增升降元素
                    A.append(pitch.split(',')[2])
                if len(A)==1 and (str(A[0])=='1' or str(A[0])=='-1'):#如果A只有一個值且值為1OR-1將該pitch升降元素改為0
                    pitch=pitch.split(',')[0]+','+pitch.split(',')[1]+',0'
                if len(A)==1:        #當A有一值時和弦升降值為0
                    i=0     
                if len(pitch.split(','))==3 and pitch.split(',')[2]=='1':#如果有上升記號
                    i+=1
                    newpitch=60+12*(i/len(dic_cp[cp[j]]))+dic_cp[cp[j]][i%len(dic_cp[cp[j]])]
                    durationType_pitch=durationType_pitch+';'+pitch.split(',')[0]+','+str(newpitch)
                elif len(pitch.split(','))==3 and pitch.split(',')[2]=='-1':#如果有下降記號
                    i+=-1
                    newpitch=60+12*(i/len(dic_cp[cp[j]]))+dic_cp[cp[j]][i%len(dic_cp[cp[j]])]
                    durationType_pitch=durationType_pitch+';'+pitch.split(',')[0]+','+str(newpitch)
                else:
                    if pitch.split(',')[1]=='0':
                        newpitch=0
                    else:
                        newpitch=60+12*(i/len(dic_cp[cp[j]]))+dic_cp[cp[j]][i%len(dic_cp[cp[j]])]
                    if durationType_pitch=='':
                        durationType_pitch=durationType_pitch+pitch.split(',')[0]+','+str(newpitch)
                    else:
                        durationType_pitch=durationType_pitch+';'+pitch.split(',')[0]+','+str(newpitch) 
                print '和弦',cp[j],'陣列第',i,'個,發聲',pitch.split(',')[1],',pitch是',newpitch,
                duration=duration+float(pitch.split(',')[0])
                print ',累積duration',duration
                if duration>=0.5:
                    j=k+1       #'換調囉'
            aa_i[key]=durationType_pitch
    k+=2
    if k%len(cp)==0:#重複CP
        k=0             
    print 'A_pattern',aa_i
    print '===========================' 


和弦 C 陣列第 0 個,發聲 1 ,pitch是 60 ,累積duration 0.125
和弦 C 陣列第 0 個,發聲 1 ,pitch是 60 ,累積duration 0.25
和弦 C 陣列第 0 個,發聲 1 ,pitch是 60 ,累積duration 0.375
和弦 C 陣列第 0 個,發聲 1 ,pitch是 60 ,累積duration 0.5
和弦 D 陣列第 0 個,發聲 1 ,pitch是 62 ,累積duration 0.625
和弦 D 陣列第 0 個,發聲 1 ,pitch是 62 ,累積duration 0.75
和弦 D 陣列第 0 個,發聲 1 ,pitch是 62 ,累積duration 0.875
和弦 D 陣列第 0 個,發聲 1 ,pitch是 62 ,累積duration 1.0
更換後的track {u'track0': u'0.125,60;0.125,60;0.125,60;0.125,60;0.125,62;0.125,62;0.125,62;0.125,62'}
和弦 Bb 陣列第 0 個,發聲 1 ,pitch是 70 ,累積duration 0.125
和弦 Bb 陣列第 0 個,發聲 1 ,pitch是 70 ,累積duration 0.25
和弦 Bb 陣列第 0 個,發聲 1 ,pitch是 70 ,累積duration 0.375
和弦 Bb 陣列第 0 個,發聲 1 ,pitch是 70 ,累積duration 0.5
和弦 E 陣列第 0 個,發聲 1 ,pitch是 64 ,累積duration 0.625
和弦 E 陣列第 0 個,發聲 1 ,pitch是 64 ,累積duration 0.75
和弦 E 陣列第 0 個,發聲 1 ,pitch是 64 ,累積duration 0.875
和弦 E 陣列第 0 個,發聲 1 ,pitch是 64 ,累積duration 1.0
更換後的track {u'track0': u'0.125,70;0.125,70;0.125,70;0.125,70;0.125,64;0.125,64;0.125,64;0.125,64'}
和弦 C 陣列第 0 個,發聲 0 ,pitch是 0 ,累積duration 0.125
和弦 C 陣列第 0 個,發